# Instrucciones:

1. Antes de comenzar, realiza una copia de este **Google Colab Notebook** en tu propia cuenta de Google Drive. De lo contrario, no se podrá guardar ningún cambio.
2. En el título, agrega tu nombre en el espacio correspondiente. 
3. Una vez terminado, exporta tus resultados en formato PDF. Pasos: Archivo > Imprimir > Imprimir en archivo.


# Python

#### 1. Crea una función llamada **search** que buscará cualquier término de búsqueda en la Base de Datos de Wikipedia y devolverá el resultado de dicha consulta. Posteriormente ejecuta lo siguiente:
- La función recibe como único parámetro de entrada: **search_term** que representa el término a buscar .
- La función tendrá como salida un resultado de tipo JSON llamado **response_content** con el contenido de respuesta de la API. 
- Imprimir el resultado en pantalla

> Input (Type String): search_term
>
> Output (Type JSON): response_content

[Documentación](https://www.mediawiki.org/wiki/API:Search#GET_request)
#### Para el siguiente ejercicio, usa la libreria **requests** de python para conectarse a la API RESTful de Wikipedia


In [1]:
try:
  import requests
  import pandas as pd
  import json
  import requests
except ImportError as eImp:
    print(f"Se produjo un error al importar las siguientes librerias: {eImp}")

def search(search_term):
  se = requests.Session()
  URL = "https://en.wikipedia.org/w/api.php"
  PARAMS = {
    "action": "query",
    "format": "json",
    "list": "search",
    "srsearch": search_term
  } 
  response_content = se.get(url=URL, params=PARAMS)
  response_content = response_content.json()
  return response_content
  
# Ejemplo
search("Big Data")

{'batchcomplete': '',
 'continue': {'sroffset': 10, 'continue': '-||'},
 'query': {'searchinfo': {'totalhits': 65901},
  'search': [{'ns': 0,
    'title': 'Big data',
    'pageid': 27051151,
    'size': 157231,
    'wordcount': 15455,
    'snippet': '<span class="searchmatch">Big</span> <span class="searchmatch">data</span> primarily refers to <span class="searchmatch">data</span> sets that are too large or complex to be dealt with by traditional <span class="searchmatch">data</span>-processing application software. <span class="searchmatch">Data</span> with many',
    'timestamp': '2023-02-15T17:29:06Z'},
   {'ns': 0,
    'title': 'Data',
    'pageid': 18985040,
    'size': 20432,
    'wordcount': 2451,
    'snippet': 'to the advent of <span class="searchmatch">big</span> <span class="searchmatch">data</span>, which usually refers to very large quantities of <span class="searchmatch">data</span>, usually at the petabyte scale. Using traditional <span class="searchmatch">data</span> an

#### 2. Construye una función llamada **get_results_table** que tomará como entrada el resultado de la función **search** y tendrá como salida un **Data Frame** de la librería Pandas. Posteriormente ejecuta lo siguiente:
- La función recibe como parámetro de entrada la salida de la función **search** 
- La función tendrá como salida un Data Frame de Pandas llamado **df**
- Elimina las columnas **ns, snippet** como regla de negocio.
- Agrega una columna llamada **search_term** que contenga el término buscado
- Imprimir el resultado en pantalla
- Guarda el resultado en un archivo **CSV** llamado **salida.csv** (Si lo ejecutas en Drive, se deberá cargar en el folder */content*)

   >Input (Type String): output_call_search
   >
   >Output (Type Pandas Data Frame): df

In [2]:
def get_results_table(output_call_search):
    columns_to_delete = ['snippet', 'ns']
    dfx = pd.DataFrame.from_records(output_call_search['query']['search'])
    dfx.drop(columns_to_delete, axis='columns', inplace=True)
    df=dfx.assign(search_term = search_term)
    df=dfx.assign(search_term = search_term)
    df.to_csv('./salida.csv')
    return df

In [3]:
search_term = "Big Data"
results = search(search_term)
get_results_table(results)

,title,pageid,size,wordcount,timestamp,search_term
0,Big data,27051151,157231,15455,2023-02-15T17:29:06Z,Big Data
1,Data,18985040,20432,2451,2023-02-18T03:04:50Z,Big Data
2,Big Data (band),41460310,8976,351,2022-04-29T02:47:48Z,Big Data
3,Data science,35458904,19299,1849,2023-02-12T02:13:45Z,Big Data
4,Big data ethics,55181525,19091,2209,2023-02-07T19:31:28Z,Big Data
5,List of big data companies,51354460,2933,320,2023-01-19T12:19:02Z,Big Data
6,Industrial big data,48415691,15260,1365,2023-02-08T05:27:14Z,Big Data
7,Data lake,46626475,8264,929,2023-01-26T16:06:03Z,Big Data
8,Data analysis,2720954,84663,9156,2023-02-16T00:57:41Z,Big Data
9,Data set,8495,8214,848,2023-02-13T02:47:46Z,Big Data


# Spark
#### **! Asegurate de tener lo siguiente.**
- Actualiza el archivo **csv**  del ejercicio anterior dentro de los archivos de este notebook (/content/file.csv).
- Ejecuta el siguiente *chunk* para instalar Spark


In [4]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 12.0 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=f4a9f0e0b9eb8e80d4c95bf0cb12bd104dad62f0f3c1c16ace28d5e3ed5fe792
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


#### 1. Crea una función llamada **main** que construya un Data Frame de Spark con base en el archivo CSV anterior. Posteriormente ejecuta lo siguiente:
- La función recibe como parámetro: **path** que es la ruta del archivo CSV.
- La función tendrá como salida un Data Frame de spark
- Elimina las columnas **ns, snippet** como regla de negocio.
- Ordena las columas de la siguiente forma: Title, Timestamp, Wordcount
- Cambia el nombre de las columnas manteniendo el mismo orden el punto anterior: Título, Date, Word Count
- Ordena la tabla del dato más reciente hasta el más antiguo mediante el campo **timestamp**
- Imprime el resultado en pantalla

> Input (Type String): path
>
> Output (Type JSON): df

In [5]:
try:
  import pandas as pd
  from pyspark.sql import SparkSession
  from pyspark.sql.functions import to_timestamp
except ImportError as eImp:
  print(f"Se produjo un error al importar las siguientes librerias: {eImp}")

spark = SparkSession.builder.appName("App - Ingeniería de Datos").getOrCreate()
    
def main(path):
  df = spark.read.csv(path,header=True)
  df = df[['title','timestamp','wordcount','pageid','size','search_term']]
  df = df.withColumnRenamed("title", "Titulo")
  df = df.withColumnRenamed("timestamp", "Date")
  df = df.withColumnRenamed("wordcount", "Word Count")
  df = df.withColumn("Date",to_timestamp("Date"))
  df = df.sort(('Date'))
  df.show()
path ='/content/salida.csv'  
main(path)
spark.stop()

+--------------------+-------------------+----------+--------+------+-----------+
|              Titulo|               Date|Word Count|  pageid|  size|search_term|
+--------------------+-------------------+----------+--------+------+-----------+
|     Big Data (band)|2022-04-29 02:47:48|       351|41460310|  8976|   Big Data|
|List of big data ...|2023-01-19 12:19:02|       320|51354460|  2933|   Big Data|
|           Data lake|2023-01-26 16:06:03|       929|46626475|  8264|   Big Data|
|     Big data ethics|2023-02-07 19:31:28|      2209|55181525| 19091|   Big Data|
| Industrial big data|2023-02-08 05:27:14|      1365|48415691| 15260|   Big Data|
|        Data science|2023-02-12 02:13:45|      1849|35458904| 19299|   Big Data|
|            Data set|2023-02-13 02:47:46|       848|    8495|  8214|   Big Data|
|            Big data|2023-02-15 17:29:06|     15455|27051151|157231|   Big Data|
|       Data analysis|2023-02-16 00:57:41|      9156| 2720954| 84663|   Big Data|
|               

# SQL
En esta sección, usarás dos conjuntos de datos de código abierto para responder unas preguntas más adelante: 
- **flights.csv**: contiene 100k registros de vuelos del año 2015 dentro de Estados Unidos. 
- **airports.csv**: catálogo de aeropuertos en Estados Unidos

Ambos datasets se encuentran disponibles en el siguiente [Enlace](https://drive.google.com/drive/folders/1vjeel7rQQs6gqLTTZKuHdePnHHSzDzfS?usp=sharing).

#### ! Asegurate de tener lo siguiente.
- Datasets alojados en /content/file.csv (si lo ejecutas en Drive)
- Ejecuta el *chunk* anterior para instalar Spark


Realiza lo siguiente:



#### **Contexto**: 
La Oficina de Estadísticas de Transporte del Departamento de Transporte de EE. UU. (DOT) rastrea el desempeño puntual de los vuelos nacionales operados por grandes compañías aéreas. La información resumida sobre el número de vuelos puntuales, retrasados, cancelados y desviados se publica en el Informe del consumidor de viajes aéreos mensual del DOT y en este conjunto de datos de retrasos y cancelaciones de vuelos de 2015.

1. Realiza un recuerdo del número de vuelos entre un aeropuerto de origen (origin_airport) y un aeropuerto de destino (destination_airport) con retraso de salida (departure_delay) pero que llegaron a tiempo (arrival_delay). Agrega el nombre del aeropuerto disponible en el campo airport de dataset de airports.

2. Calcular el promedio diario de tiempo de vuelo (air_time) entre la ruta de San Francisco (origin_airport = 'SFO') y Los Ángeles (destination_airport='LAX')

3. Encontrar la aerolínea y el motivo de cancelación más frecuente en los vuelos que fueron cancelados en el año 2015. La columna FREQUENCY indica el motivo de cancelación más frecuente para cada aerolínea, y puede tomar uno de los siguientes valores: 
- A: que significa cancelación por razones del operador de la aerolínea .

- B: que significa cancelación por razones técnicas.

- C: que significa cancelación por razones de seguridad.

- D: que significa cancelación debido a las condiciones climáticas. 


> Debe mostrar las aerolíneas que tuvieron la mayor frecuencia de cancelaciones por cada uno de estos motivos. Si hay varias aerolíneas con la misma frecuencia máxima, la consulta debe mostrar todas ellas. Los resultados deben estar ordenados en orden descendente según la frecuencia máxima de cancelaciones.



**Para ambas consultas, imprime el resultado en pantalla**

In [26]:
try:
  import pandas as pd
  from pyspark.sql import SparkSession
except ImportError as eImp:
  print(f"Se produjo un error al importar las siguientes librerias: {eImp}")

spark = SparkSession.builder.appName("App - Ingeniería de Datos").getOrCreate()
def fly():
  flights = spark.read.csv('/content/flights.csv', header=True)
  airlines = spark.read.csv('/content/airports.csv', header=True)
  flights.createOrReplaceTempView('flights')
  airlines.createOrReplaceTempView('airlines')
  query = """
  SELECT a1.AIRPORT AS ORIGIN_AIRPORT_NAME, a2.AIRPORT AS DESTINATION_AIRPORT_NAME, COUNT(*) AS NUMBERFLIGHTS 
  FROM flights f
  JOIN airlines a1 ON f.ORIGIN_AIRPORT = a1.IATA_CODE
  JOIN airlines a2 ON f.DESTINATION_AIRPORT = a2.IATA_CODE
  WHERE f.DEPARTURE_DELAY > 0 AND f.ARRIVAL_DELAY <= 0 
  GROUP BY ORIGIN_AIRPORT_NAME, DESTINATION_AIRPORT_NAME 
  ORDER BY NUMBERFLIGHTS DESC
  """
  query2= """
  SELECT AVG(AIR_TIME) AS promedio_diario_tiempo
  FROM flights
  WHERE ORIGIN_AIRPORT = 'SFO'
  AND DESTINATION_AIRPORT = 'LAX'
  AND CANCELLED = 0
  AND AIR_TIME > 0
  """
  query3 ="""
  SELECT AIRLINE, MAX(CANCELLATION_REASON) AS FREQUENCY 
  FROM flights 
  WHERE CANCELLED = 1 AND YEAR = 2015 
  GROUP BY AIRLINE 
  ORDER BY FREQUENCY DESC;
  """
  view_query = spark.sql(query)
  view_query.show()
  view_query2 = spark.sql(query2)
  view_query2.show()
  view_query3 = spark.sql(query3)
  view_query3.show()
fly()
spark.stop()

+--------------------+------------------------+-------------+
| ORIGIN_AIRPORT_NAME|DESTINATION_AIRPORT_NAME|NUMBERFLIGHTS|
+--------------------+------------------------+-------------+
|Dallas/Fort Worth...|    Los Angeles Inter...|           31|
|San Francisco Int...|    Chicago O'Hare In...|           30|
|John F. Kennedy I...|    Los Angeles Inter...|           29|
|Newark Liberty In...|    San Francisco Int...|           29|
|Chicago O'Hare In...|    Los Angeles Inter...|           28|
|Los Angeles Inter...|    San Francisco Int...|           27|
|George Bush Inter...|    San Francisco Int...|           27|
|Chicago O'Hare In...|    LaGuardia Airport...|           26|
|Fort Lauderdale-H...|    Hartsfield-Jackso...|           26|
|Los Angeles Inter...|    Chicago O'Hare In...|           24|
|Los Angeles Inter...|    John F. Kennedy I...|           24|
|San Francisco Int...|    John F. Kennedy I...|           23|
|San Francisco Int...|    Los Angeles Inter...|           23|
|Chicago